## Logistic Regression and SVM

CS184A/284A Assignment 2<br>
Fall 2016


In this assignment, we will walk you through the process of implementing
- Logistic regression
- Gradient descent 
- Subgr
- Support Vector Machines (SVM)

The purpose of this assignment is to familiarize you with basic knowledge about linear regression, including optimization and cross-validation, and help you gain proficiency in writing efficient code.

** Please don't add or remove any code cells, as it might break our automatic grading system and affect your grade. **


**Honor Code:** I hereby agree to abide the UCI Honor Code and that of the Computer Science Department, promise that the submitted assignment is my own work, and understand that my code is subject to plagiarism test.

**Signature**: *(Stanislav Listopad)*

In [50]:
# Run some setup code for this notebook. Don't modify anything in this cell.
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 6)
import random
from sklearn import linear_model
from sklearn import datasets, linear_model, cross_validation, metrics, preprocessing
random.seed(1)

## Pima Indians diabetes dataset
https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes

Attribute Information:

1. Number of times pregnant 
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test 
3. Diastolic blood pressure (mm Hg) 
4. Triceps skin fold thickness (mm) 
5. 2-Hour serum insulin (mu U/ml) 
6. Body mass index (weight in kg/(height in m)^2) 
7. Diabetes pedigree function 
8. Age (years) 
9. Class variable (0 or 1) 

In [67]:
# load pima indians dataset
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
diabetes_X = dataset[:,0:8]
diabetes_Y = dataset[:,8]

In [68]:
# rescale each feature to be mean 0 and var 1
diabetes_X_scale = preprocessing.scale(diabetes_X)

In [69]:
# add a constant of 1 to the first column of a matrix
def add_constant(x):
    """ Add a column of constant 1 """
    x = np.concatenate( (np.ones((x.shape[0],1)), x), axis=1)
    return x

In [70]:
diabetes_X_scale_aug = add_constant(diabetes_X_scale)

In [71]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(\
                            diabetes_X_scale_aug, diabetes_Y, test_size=0.3, random_state=1)

## Define a score function to evaluate prediction accuracy 

In [56]:
def score(y_pred, y_obs):
    """
    y_obs:  vector of observed labels
    y_pred: vector of predicted labels

    returns the percentage of predicted labels matching the observed labels. 
    """
    
    ### YOUR CODE HERE
    
    score = 0
    index = 0
    size = y_pred.shape[0]
    while(index < size):
        if(y_pred[index] == y_obs[index]):
            score +=1
        index +=1 
    score = score / float(size)

    
    ### END OF YOUR CODE
    
    return score

In [57]:
# Test this function
print "=== For autograder ==="
score(np.ones(y_test.shape), y_test) 

# should return  0.36796536796536794

=== For autograder ===


0.36796536796536794

## Logistic Regression

* Model:    $$P(y=1|x) = g(\theta^T x)$$ 
where $g(z) = \frac{1}{1+e^{-z}}$ is the sigmoid function.
    The model predicts y=1 if $P(y=1|x)>0.5$ and 0 otherwise.

* Cost function:
$$J(\theta) = - \sum_{i=1}^m [y^{(i)} \log(P(y^{(i)}=1)|x^{(i)})+(1-y^{(i)}) \log(P(y^{(i)}=0)|x^{(i)})]$$
* In maxtir representation: let $y$ be an $m$-dimensional vector and $X$ be an $m$ by $n+1$ matrix.
$$J(\theta) = - \frac{1}{m}[y^T \log g(X\theta) + (1-y)^T \log (1- g(X\theta))]$$

* The gradient of $J(\theta)$ w.r.t. $\theta$ is
$$\nabla J(\theta) = \frac{1}{m} X^T(g(X\theta)-y)$$

** Now define a function which receives $X$, $y$, $\theta$ as input and outputs both cost and gradient** 

In [58]:
def sigmoid(x):
    """ Sigmoid function """
    ###################################################################
    # Compute the sigmoid function for the input here.                #
    ###################################################################
    
    ### YOUR CODE HERE

    f = np.exp(-x)
    f = np.add(f, 1)
    f = np.divide(1, f)
    ### END YOUR CODE
    
    return f

# Test this function
print "=== For autograder ==="
print sigmoid(np.array([-1., 0, 1.]))
# should produce [ 0.26894142  0.5         0.73105858]

=== For autograder ===
[ 0.26894142  0.5         0.73105858]


##  Gradient descent

Next we walk through steps of implementing gradient descent algorithm

In [59]:
def logistic_regression_cost_grad(X, y, theta):
    """
    calculate cost and grad of logistic regression cost function
    X:  m x (n+1)  (m: number of samples, n: number of feature dimension)
    y:  m x 1  (target)
    beta: (n+1),   (beta_0 is the intercept)
    output: cost, grad   where grad should be an one-dimensional array with size n+1. 
    """    
    
    ### YOUR CODE HERE
    x_1 = sigmoid(np.dot(X, theta))
    m = X.shape[0]
    x_2 = np.dot(np.transpose(y), np.log(x_1))
    x_3 = np.dot(np.transpose(np.subtract(1, y)), np.log(np.subtract(1, x_1)))
    cost = np.dot((-1.0/m), (x_2 + x_3))
    
    x_2 = np.subtract(x_1, y)
    x_3 = np.dot(np.transpose(X), x_2)
    grad = np.dot((1.0/m), x_3)
    
    
    ### END YOUR CODE
    
    return cost, grad

# Test this function
print "=== For autograder ==="
print logistic_regression_cost_grad(np.array([[1.,2.],[2, 3],[3,4]]), np.array([0,1,1]), np.array([1,-1]))  
# should produce (0.97992835418488955, array([-1.12878382, -1.52650907]))

=== For autograder ===
(0.97992835418488955, array([-1.12878382, -1.52650907]))


### Implement gradient descent algorithm

In [60]:
def sgd(f, x0, step, iterations, PRINT_EVERY=10):
    """ Stochastic Gradient Descent """
    ###################################################################
    # Implement the stochastic gradient descent method in this        #
    # function.                                                       #
    # Inputs:                                                         #
    #   - f: the function to optimize, it should take a single        #
    #        argument and yield two outputs, a cost and the gradient  #
    #        with respect to the arguments                            #
    #   - x0: the initial point to start SGD from                     #
    #   - step: the step size for SGD                                 #
    #   - iterations: total iterations to run SGD for                 #
    #   - PRINT_EVERY: specifies every how many iterations to output  #
    # Output:                                                         #
    #   - x: the parameter value after SGD finishes                   #
    ###################################################################
    
    # Anneal learning rate every several iterations
    #
    ANNEAL_EVERY = 10000    

    x = x0    
    for iter in xrange(iterations):

        ### YOUR CODE HERE        
        cost, grad = f(x)
        x -= step*grad                
        ### END YOUR CODE      
        
        if iter % PRINT_EVERY == 0:
            print "iter: %d cost: %f" % (iter, cost)               
      
        if iter % ANNEAL_EVERY == 0:
            step *= 0.5
            
        #plt.plot(iter, cost, '.b')            
    return x

## Run batch gradient descent on the diabetes training data

In [61]:
# set up parameters for running sgd,  do not change these parameters
np.random.seed(1)
theta0 = np.random.randn(X_train.shape[1],)  # randomly initialize theta
step = 0.1     # step size
niter = 5000   # number of iterations

## Now run gradient descent on the diabetes training data using linear_regression_cost_grad

### YOUR CODE HERE   
index = 0
while (index < niter):
    cost, grad = logistic_regression_cost_grad(X_train, y_train, theta0)
    theta0 = np.subtract(theta0, np.multiply(step, grad))
    index += 1
theta_batch = theta0
### END YOUR CODE

In [73]:
# calcluate prediction accuracy of the trained logistic regression model

### YOUR CODE HERE
y_pred = np.zeros(y_train.shape[0])

index = 0
while(index < X_train.shape[0]):
    if(sigmoid(np.dot(np.transpose(theta_batch), X_train[index, :])) > 0.5):
        y_pred[index] = 1
    else:
        y_pred[index] = 0
    index += 1
train_accuracy = score(y_pred, y_train)


index = 0
y_pred = np.zeros(y_test.shape[0])

while(index < X_test.shape[0]):
    if(sigmoid(np.dot(np.transpose(theta_batch), X_test[index, :])) > 0.5):
        y_pred[index] = 1
    else:
        y_pred[index] = 0
    index += 1
test_accuracy = score(y_pred, y_test)
### END OF YOUR CODE


### END OF YOUR CODE

print "=== For autograder ==="
print train_accuracy, test_accuracy
# should retrun 0.776536312849 0.78354978355

=== For autograder ===
0.776536312849 0.78354978355


## SVM 

- Linear classifier:  Assign $y=+1$ if $w^Tx+b>0$ and $y=-1$ otherwise. $w\in R^n$ and $b\in R$ are parameters of the model.

- Hinge loss function: $$ J(w,b) = \sum_{i=1}^m \max\{0, 1- y^{(i)}(w^Tx^{(i)}+b)\} + \frac{\alpha}{2} \|w\|_2^2 $$
where $\alpha$ is the regularization parameter
- In this assignment, we will implement a subgradient method to solve SVM optimization.

In [65]:
# change data according to SVM format
X_train_svm = X_train[:,1:]   # excluding constant 
y_train_svm = y_train 
y_train_svm[y_train==0] = -1

In [34]:
# random initialization of w and b
np.random.seed(1)
theta0 = np.random.randn(X_train.shape[1],)  # randomly initialize theta
w = theta0[1:]
b = theta0[0]

In [35]:
# now implement a function that compute the cost and sub-gradient of the SVM cost function
def SVM_cost_grad(X, y, w, b, alpha):
    """
    calculate cost and grad of mean square error cost function
    X:  (m,n)  (m: number of samples, n: number of feature dimension)
    y:  (m,)  target (+1, -1)
    w:  (n,)
    b:  scalar 
    alpha: scalar
    output: cost, grad
            grad: one-dimensional array with shape (n+1,), gradidents w.r.t. w and b
            with the FIRST component being gradient w.r.t. b

    """
    
    ### YOUR CODE HERE    

    m = X.shape[0]
    n = X.shape[1]
    index = 0
    sum = 0
    while(index < m):
        x_1 = np.dot(np.transpose(w), X[index, :])
        x_2 = np.add(x_1, b)
        x_3 = np.dot(y[index], x_2)
        x_4 = np.subtract(1, x_3)
        x_5 = np.maximum(0, x_4)
        sum += x_5
        index += 1
    cost = sum + np.dot((alpha/2.0), np.linalg.norm(np.square(w), 1))
    
    index = 0
    sumW = np.zeros(n)
    sumb = 0
    
    grad = np.zeros(n+1)
    
    while(index < m):
        x_1 = np.dot(np.transpose(w), X[index, :])
        x_2 = np.add(x_1, b)
        x_3 = np.dot(y[index], x_2)
        if(x_3 < 1.0):
            x_1 = np.dot(-1, y[index])
            x_2 = np.dot(x_1, X[index, :])
            sumW = np.add(sumW, x_2)
            sumb += x_1
        index += 1
    sumW = np.add(sumW, np.dot(alpha, w))
    grad[1:(n+1)] = sumW
    grad[0] = sumb
        
    
    ### END YOUR CODE
    
    return cost, grad

# Test this function
print "=== For autograder ==="

SVM_cost_grad(X_train_svm,y_train_svm, w, b, 1.0)
# (1278.3956978213123, array([ 223.        , -102.90084914, -208.88830549,  -46.54214938,
#         -42.45648627, -163.36303984,  -47.83554692, -105.40353328,
#         -82.36221523]))

=== For autograder ===


(1278.3956978213123,
 array([ 223.        , -102.90084914, -208.88830549,  -46.54214938,
         -42.45648627, -163.36303984,  -47.83554692, -105.40353328,
         -82.36221523]))

In [36]:
# set up parameters for running sgd,  do not change these parameters
alpha = 10
step = 0.001    # step size
niter = 1000   # number of iterations

## Using subgradient method to find optimal paramters of linear SVM
## Now run gradient descent on the diabetes training data using SVM_cost_grad

### YOUR CODE HERE   
index = 0
while (index < niter):
    cost, grad = SVM_cost_grad(X_train_svm, y_train_svm, w, b, alpha)
    theta0 = np.subtract(theta0, np.multiply(step, grad))
    w = theta0[1:]
    b = theta0[0]
    index += 1
theta_svm = theta0

### END YOUR CODE

In [37]:
print "=== For autograder ==="
print theta_svm
# should produce [-0.70665464  0.24140978  0.83001714 -0.20227141 -0.12249842 -0.03130514
#  0.54549111  0.16816575  0.09153396]

=== For autograder ===
[-0.70565464  0.24119426  0.8316116  -0.20231241 -0.11929713 -0.0271685
  0.54491416  0.16806344  0.09385541]


In [38]:
# calcluate prediction accuracy of the trained SVM model

### YOUR CODE HERE

#theta_svm = [-0.70665464,  0.24140978,  0.83001714, -0.20227141, -0.12249842, -0.03130514, 0.54549111,  0.16816575,  0.09153396]

y_svm_pred = np.zeros(y_train_svm.shape[0])

index = 0
while(index < X_train_svm.shape[0]):
    if(sigmoid(np.add(theta_svm[0], np.dot(np.transpose(theta_svm[1::]), X_train_svm[index, :]))) > 0.5):
        y_svm_pred[index] = 1
    else:
        y_svm_pred[index] = -1
    index += 1
train_accuracy_svm = score(y_svm_pred, y_train_svm)


# change data according to SVM format
X_test_svm = X_test[:,1:]   # excluding constant 
y_test_svm = y_test 
y_test_svm[y_test==0] = -1

index = 0
y_svm_pred = np.zeros(y_test_svm.shape[0])

while(index < X_test_svm.shape[0]):
    if(sigmoid(np.add(theta_svm[0], np.dot(np.transpose(theta_svm[1::]), X_test_svm[index, :]))) > 0.5):
        y_svm_pred[index] = 1
    else:
        y_svm_pred[index] = -1
    index += 1
test_accuracy_svm = score(y_svm_pred, y_test_svm)
### END OF YOUR CODE

print "=== For autograder ==="
print train_accuracy_svm, test_accuracy_svm
# should retrun 0.780260707635 0.792207792208

=== For autograder ===
0.778398510242 0.792207792208
